In [1]:
import numpy as np
import pandas as pd
from keras.models import load_model

Using TensorFlow backend.


In [2]:
test_features = pd.read_csv("dengue_features_test.csv", index_col=[0,1,2]) 
train_features = pd.read_csv("dengue_features_train.csv", index_col=[0,1,2]) 
train_labels = pd.read_csv("dengue_labels_train.csv", index_col=[0,1,2]) 
submission_format = pd.read_csv("submission_format.csv", index_col=[0,1,2]) 

In [16]:
select_features = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c', 
                 'station_min_temp_c', 'week_start_date']

## Only IQ Columns

In [17]:
# Seperate data for San Juan
iq_train_features = train_features.loc['iq']
iq_train_labels = train_labels.loc["iq"]
iq_test_features = test_features.loc["iq"]

In [18]:
#removing NaNs
iq_train_features.fillna(0, inplace=True)
iq_test_features.fillna(0, inplace=True)

c:\users\milla\projects\databears\lib\site-packages\pandas\core\frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [19]:
iq_train_features.set_index("week_start_date")
iq_test_features.set_index("week_start_date")

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
week_start_date,,,,,,,,,,,,,,,,,,,,
2010-07-02,0.183783,0.142500,0.225129,0.150214,82.29,297.648571,299.214286,296.118571,307.9,293.7,34.11,92.581429,82.29,17.654286,9.428571,27.440,10.760,33.8,21.5,11.2
2010-07-09,0.291657,0.272267,0.330700,0.320914,25.30,298.224286,299.464286,294.831429,307.7,292.2,9.10,83.885714,25.30,16.320000,10.157143,27.025,9.625,33.0,21.2,8.9
2010-07-16,0.208543,0.366457,0.212629,0.255514,62.14,297.955714,298.964286,296.417143,306.4,292.5,61.09,92.057143,62.14,18.030000,9.557143,26.950,10.350,33.4,21.6,22.6
2010-07-23,0.089286,0.063214,0.122057,0.081957,47.80,295.715714,296.642857,293.638571,306.4,289.3,19.60,88.970000,47.80,15.394286,7.828571,26.900,9.700,33.3,14.2,4.8
2010-07-30,0.306100,0.327683,0.250086,0.267914,56.30,298.502857,300.000000,293.892857,308.1,290.5,18.93,78.610000,56.30,15.468571,11.771429,27.050,11.850,33.5,16.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-05-28,0.301471,0.380029,0.280629,0.383186,41.12,297.774286,298.964286,295.638571,305.5,292.7,67.60,89.990000,41.12,17.185714,10.100000,27.400,9.050,32.6,21.8,33.0
2013-06-04,0.247600,0.296343,0.285371,0.350357,71.52,297.167143,298.328571,295.845714,306.3,291.6,45.70,93.891429,71.52,17.448571,9.657143,27.520,10.720,33.8,21.4,68.0
2013-06-11,0.238729,0.251029,0.252586,0.249771,78.96,295.831429,296.607143,294.894286,304.6,290.7,45.22,94.967143,78.96,16.410000,7.385714,27.200,10.075,32.6,21.6,93.2


In [20]:
#removing NaNs - test

iq_train_features.isnull().sum()
# iq_train_features

week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
dtype: int64

In [21]:
iq_train_labels["week_start_date"] = iq_train_features["week_start_date"]

In [22]:
iq_train_labels.set_index("week_start_date")

,total_cases
week_start_date,
2000-07-01,0
2000-07-08,0
2000-07-15,0
2000-07-22,0
2000-07-29,0
...,...
2010-05-28,5
2010-06-04,8
2010-06-11,1


In [23]:
iq_train_features = iq_train_features[select_features]

In [24]:
iq_train_features

reanalysis_specific_humidity_g_per_kg  \
year weekofyear                                          
2000 26                                      16.651429   
     27                                      16.862857   
     28                                      17.120000   
     29                                      14.431429   
     30                                      15.444286   
...                                                ...   
2010 21                                      18.485714   
     22                                      18.070000   
     23                                      17.008571   
     24                                      16.815714   
     25                                      17.355714   

                 reanalysis_dew_point_temp_k  station_avg_temp_c  \
year weekofyear                                                    
2000 26                           295.184286           26.400000   
     27                           295.358571           26.900000   
     28                           295.622857           26.800000   
     29                           292.797143           25.766667   
     30                           293.957143           26.600000   
...                                      ...                 ...   
2010 21                           296.825714           28.633333   
     22                           296.452857           27.433333   
     23                           295.501429           24.400000   
     24                           295.324286           25.433333   
     25                           295.807143           27.475000   

                 station_min_temp_c week_start_date  
year weekofyear                                      
2000 26                        20.7      2000-07-01  
     27                        20.8      2000-07-08  
     28                        20.7      2000-07-15  
     29                        14.7      2000-07-22  
     30                        19.1      2000-07-29  
...                             ...             ...  
2010 21                        22.4      2010-05-28  
     22                        21.7      2010-06-04  
     23                        19.2      2010-06-11  
     24                        21.0      2010-06-18  
     25                        22.2      2010-06-25  

[520 rows x 5 columns]

In [25]:
# data normalization
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler(feature_range=(0,1))
sc_Y = MinMaxScaler(feature_range=(0,1))
iq_train_features_scaled = sc_X.fit_transform(iq_train_features.drop(columns = ["week_start_date"]))
iq_train_labels_scaled = sc_Y.fit_transform(np.reshape(iq_train_labels["total_cases"].tolist(), (520, 1)))

In [26]:
iq_train_features_scaled

array([[0.81379599, 0.98905775, 0.85714286, 0.8553719 ],
       [0.82412902, 0.98964172, 0.87337662, 0.85950413],
       [0.83669622, 0.99052725, 0.87012987, 0.8553719 ],
       ...,
       [0.83125044, 0.99012038, 0.79220779, 0.79338843],
       [0.82182504, 0.98952684, 0.82575758, 0.8677686 ],
       [0.84821616, 0.99114472, 0.89204545, 0.91735537]])

In [27]:
iq_train_labels_scaled.shape

(520, 1)

In [28]:
iq_train_labels

total_cases week_start_date
year weekofyear                             
2000 26                    0      2000-07-01
     27                    0      2000-07-08
     28                    0      2000-07-15
     29                    0      2000-07-22
     30                    0      2000-07-29
...                      ...             ...
2010 21                    5      2010-05-28
     22                    8      2010-06-04
     23                    1      2010-06-11
     24                    1      2010-06-18
     25                    4      2010-06-25

[520 rows x 2 columns]

In [29]:
#incoperating time steps into data

X_train = []
y_train = []
for i in range(0, len(iq_train_features_scaled)):
    X_train.append(iq_train_features_scaled[i,:])
    y_train.append(iq_train_labels_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

In [30]:
X_train.shape

(520, 1, 4)

In [31]:
X_train[1:].shape

(519, 1, 4)

In [33]:
#Model

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train[:400],y_train[:400],epochs=100,batch_size=32, 
          validation_data=(X_train[400:], y_train[400:]))
model.save('LSTM_IQ_selectfeatures.h5')

Train on 400 samples, validate on 120 samples
Epoch 1/100
400/400 [==============================] - 1s 3ms/step - loss: 0.0092 - val_loss: 0.0090
Epoch 2/100
400/400 [==============================] - 0s 129us/step - loss: 0.0090 - val_loss: 0.0092
Epoch 3/100
400/400 [==============================] - 0s 139us/step - loss: 0.0088 - val_loss: 0.0092
Epoch 4/100
400/400 [==============================] - 0s 153us/step - loss: 0.0084 - val_loss: 0.0088
Epoch 5/100
400/400 [==============================] - 0s 141us/step - loss: 0.0086 - val_loss: 0.0090
Epoch 6/100
400/400 [==============================] - 0s 154us/step - loss: 0.0085 - val_loss: 0.0090
Epoch 7/100
400/400 [==============================] - 0s 142us/step - loss: 0.0084 - val_loss: 0.0090
Epoch 8/100
400/400 [==============================] - 0s 149us/step - loss: 0.0084 - val_loss: 0.0090
Epoch 9/100
400/400 [==============================] - 0s 140us/step - loss: 0.0084 - val_loss: 0.0090
Epoch 10/100
400/400 [=======

400/400 [==============================] - 0s 158us/step - loss: 0.0078 - val_loss: 0.0086
Epoch 80/100
400/400 [==============================] - 0s 160us/step - loss: 0.0076 - val_loss: 0.0087
Epoch 81/100
400/400 [==============================] - 0s 196us/step - loss: 0.0080 - val_loss: 0.0086
Epoch 82/100
400/400 [==============================] - 0s 162us/step - loss: 0.0078 - val_loss: 0.0087
Epoch 83/100
400/400 [==============================] - 0s 190us/step - loss: 0.0079 - val_loss: 0.0087
Epoch 84/100
400/400 [==============================] - 0s 203us/step - loss: 0.0081 - val_loss: 0.0087
Epoch 85/100
400/400 [==============================] - 0s 177us/step - loss: 0.0079 - val_loss: 0.0087
Epoch 86/100
400/400 [==============================] - 0s 170us/step - loss: 0.0079 - val_loss: 0.0087
Epoch 87/100
400/400 [==============================] - 0s 192us/step - loss: 0.0078 - val_loss: 0.0087
Epoch 88/100
400/400 [==============================] - 0s 190us/step - loss:

In [34]:
# if model exist, just load it
# model = load_model('LSTM_IQ.h5')

In [35]:
iq_test_features = iq_test_features[select_features]

In [36]:
#pre processing for test data
iq_test_features_scaled = sc_X.fit_transform(iq_test_features.drop(columns = ["week_start_date"]))

In [37]:
iq_test_features_scaled.shape

(156, 4)

In [38]:
X_test = []
for i in range(0, len(iq_test_features_scaled)):
    X_test.append(iq_test_features_scaled[i,:])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [39]:
predictions = model.predict(X_test)

In [40]:
predictions

array([[ 0.04162979],
       [ 0.01105211],
       [ 0.05289138],
       [ 0.00352646],
       [-0.00097491],
       [ 0.01936022],
       [-0.02586402],
       [ 0.01333362],
       [-0.0209331 ],
       [ 0.02208589],
       [ 0.03116725],
       [ 0.00676805],
       [ 0.03504163],
       [-0.00095301],
       [-0.00225962],
       [ 0.02467635],
       [ 0.0486175 ],
       [ 0.06430019],
       [ 0.09612674],
       [ 0.0520609 ],
       [ 0.05879801],
       [ 0.07484567],
       [ 0.06375532],
       [ 0.07980733],
       [ 0.05074707],
       [ 0.01397848],
       [ 0.03920523],
       [ 0.01510333],
       [ 0.04340373],
       [ 0.04390919],
       [ 0.00438455],
       [ 0.03696765],
       [ 0.05198462],
       [ 0.05888041],
       [ 0.06492442],
       [ 0.06940386],
       [ 0.05103116],
       [ 0.05026357],
       [ 0.06249081],
       [ 0.06564784],
       [ 0.07061638],
       [ 0.04951957],
       [ 0.0468695 ],
       [ 0.06156121],
       [ 0.07453007],
       [ 0

In [41]:
predictions = sc_Y.inverse_transform(predictions)

In [42]:
iq_submission = submission_format.loc["iq"]
# sj_submission["total_cases"] = predictions
# sj_submission = predictions_submission.reset_index()
# sj_submission.insert(0, 'city', 'sj')

# predictions.shape
iq_submission

total_cases
year weekofyear             
2010 26                    0
     27                    0
     28                    0
     29                    0
     30                    0
...                      ...
2013 22                    0
     23                    0
     24                    0
     25                    0
     26                    0

[156 rows x 1 columns]

In [43]:
iq_submission["total_cases"] = predictions

In [44]:
iq_submission.insert(0, 'city', 'iq')

In [45]:
iq_submission.reset_index()

,year,weekofyear,city,total_cases
0,2010,26,iq,4.829056
1,2010,27,iq,1.282045
2,2010,28,iq,6.135400
3,2010,29,iq,0.409069
4,2010,30,iq,-0.113090
...,...,...,...,...
151,2013,22,iq,3.514374
152,2013,23,iq,4.384070
153,2013,24,iq,1.476531
154,2013,25,iq,-1.589515


In [46]:
iq_submission.to_csv("LSTMiq_selectfeatures.csv")